In [17]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import matplotlib.pyplot as plt
import scipy.stats as st

In [18]:
URL = 'https://en.wikipedia.org/wiki/List_of_best-selling_music_artists'
NUM_SALES_TABLES = 6

In [19]:
#Receive page content and store in the 'page' variable
page = requests.get(URL)

#Use BS library to to parse HTML, and specify parser of html
soup = BeautifulSoup(page.text, 'html')

In [20]:
#Search for all html table elements in 'soup' and select all 6 tables
sales_tables = soup.find_all('table')[:NUM_SALES_TABLES] 

In [21]:
#Parse through the HTML file for all th
headers = sales_tables[0].find_all('th')  
header_table = [header.text.strip() for header in headers][:7]

In [22]:
# Append all tr element to data_rows
data_rows = []
for t in sales_tables:
    table_rows = t.find_all('tr')
    data_rows += table_rows

data_rows

[<tr>
 <th scope="col" style="width:130px;">Artist
 </th>
 <th scope="col" style="width:122px;">Country
 </th>
 <th scope="col" style="width:122px;">Period active
 </th>
 <th scope="col" style="width:122px;">Release-year of first charted record
 </th>
 <th scope="col" style="width:122px;">Genre
 </th>
 <th scope="col" style="width:325px;"><a class="mw-redirect" href="/wiki/Music_recording_sales_certification" title="Music recording sales certification">Total certified units</a><br/><small>(<a href="/wiki/List_of_music_recording_certifications" title="List of music recording certifications">from available markets</a>)</small><sup class="reference" id="cite_ref-cert_12-0"><a href="#cite_note-cert-12">[b]</a></sup>
 </th>
 <th scope="col" style="width:150px;">Claimed sales
 </th></tr>,
 <tr>
 <th scope="row"><a href="/wiki/The_Beatles" title="The Beatles">The Beatles</a>
 </th>
 <td>United Kingdom
 </td>
 <td>1960–1970<sup class="reference" id="cite_ref-Allmusic:_The_Beatles_(Biography)_1

In [23]:
#Initialize empty list
total_data_array = []

#Iterate through column_data starting frmo the second row
for row in data_rows:
    
    # Extract text from 'td' elements
    td_elements = row.find_all('td')
    if td_elements:    # if data row, not header row
        td_data = [data.text.strip() for data in td_elements][0:4] + [data.text.strip() for data in td_elements][5:]
        
        # Extract text from the first 'a' element
        a_element = row.find('a')
        a_data = a_element.text.strip() if a_element else ""

        #Extract text from first 'li' element
        li_element = row.find_all('li')[:1]
        li_data = [data.text.strip() for data in li_element]
        
        # Combine 'td', 'a' and 'li' data
        combined_data = [a_data]  + td_data[0:4] + li_data + td_data[4:]

        #Append the combined data for each row to total_data_array
        total_data_array.append(combined_data)

print(total_data_array)


[['The Beatles', 'United Kingdom', '1960–1970[11]', '1962[11]', 'Rock / pop[11]', 'US: 217.250 million[12]', '600 million[43][44]500 million[45]'], ['Elvis Presley', 'United States', '1953–1977[46]', '1956[46]', 'Rock and roll / pop / country[46]', 'US: 199.650 million[12]', '500 million[56]'], ['Michael Jackson', 'United States', '1964–2009[57]', '1971[57]', 'Pop / rock / dance / soul / R&B[57]', 'US: 177.3 million[12]', '400 million[70]'], ['Elton John', 'United Kingdom', '1962–present[71]', '1970[71]', 'Pop / rock[71]', 'US: 138.050 million[12]', '300 million[75][76]250 million[77][78]'], ['Queen', 'United Kingdom', '1971–present[79]', '1973[79]', 'Rock[79]', 'US: 97.7 million[12]', '300 million[82][83]250 million[84]'], ['Madonna', 'United States', '1979–present[85]', '1983[85]', 'Pop / dance / electronica[85]', 'US: 87.175 million[12]', '300 million[92][93]250 million[94]'], ['Led Zeppelin', 'United Kingdom', '1968–1980[95]', '1969[95]', 'Hard rock / blues rock / folk rock[95]', '

In [24]:
#Initialize empty list
cleaned_data = []

for inner_list in total_data_array:
    cleaned_inner_list = []
    for item in inner_list:
        # Remove square brackets and extra characters
        cleaned_item = re.sub(r'\[.*\]', '', item)
        # Remove any leading/trailing whitespace
        cleaned_item = cleaned_item.strip()
        cleaned_inner_list.append(cleaned_item)
    cleaned_data.append(cleaned_inner_list)

# Print the cleaned data
for item in cleaned_data:
    print(item)


['The Beatles', 'United Kingdom', '1960–1970', '1962', 'Rock / pop', 'US: 217.250 million', '600 million']
['Elvis Presley', 'United States', '1953–1977', '1956', 'Rock and roll / pop / country', 'US: 199.650 million', '500 million']
['Michael Jackson', 'United States', '1964–2009', '1971', 'Pop / rock / dance / soul / R&B', 'US: 177.3 million', '400 million']
['Elton John', 'United Kingdom', '1962–present', '1970', 'Pop / rock', 'US: 138.050 million', '300 million']
['Queen', 'United Kingdom', '1971–present', '1973', 'Rock', 'US: 97.7 million', '300 million']
['Madonna', 'United States', '1979–present', '1983', 'Pop / dance / electronica', 'US: 87.175 million', '300 million']
['Led Zeppelin', 'United Kingdom', '1968–1980', '1969', 'Hard rock / blues rock / folk rock', 'US: 115.1 million', '300 million']
['Rihanna', 'Barbados', '2003–present', '2005', 'R&B / pop / dance / hip-hop', 'US: 259.550 million', '250 million']
['Pink Floyd', 'United Kingdom', '1965–1996, 2005, 2012–2014', '196

In [25]:
#Create a new df using the 'cleaned_data' lists with headers from 'header_table'
df = pd.DataFrame(cleaned_data, columns=header_table)
df

,Artist,Country,Period active,Release-year of first charted record,Genre,Total certified units(from available markets)[b],Claimed sales
0,The Beatles,United Kingdom,1960–1970,1962,Rock / pop,US: 217.250 million,600 million
1,Elvis Presley,United States,1953–1977,1956,Rock and roll / pop / country,US: 199.650 million,500 million
2,Michael Jackson,United States,1964–2009,1971,Pop / rock / dance / soul / R&B,US: 177.3 million,400 million
3,Elton John,United Kingdom,1962–present,1970,Pop / rock,US: 138.050 million,300 million
4,Queen,United Kingdom,1971–present,1973,Rock,US: 97.7 million,300 million
...,...,...,...,...,...,...,...
116,Bob Marley,Jamaica,1962–1981,1975,Reggae,US: 21.850 million,75 million
117,The Police,United Kingdom,1977–19862007–2008,1978,Pop / rock,US: 23.650 million,75 million
118,Barry Manilow,United States,1973–present,1973,Pop / soft rock,US: 33.3 million,75 million
119,Kiss,United States,1972–present,1974,Hard rock / heavy metal,US: 26 million,75 million


In [26]:
#Change first row index to 1
df.index = range(1, len(df) + 1)

#Drop all rows with null values 
df.dropna(how='any', inplace=True)

#Change header titles
artist_df = df.rename(columns={'Total certified units(from available markets)[b]':'US Units Sold', 'Claimed sales':'Total Claimed Sales'})

In [27]:
# Define a function to clean and convert the values
def clean_and_convert(value):
    # Remove non-numeric characters and convert to lower case
    cleaned_value = re.sub(r'[^0-9.]', '', value).lower()
    
    # Check for common text representations and convert to numeric
    if 'million' in cleaned_value:
        return float(cleaned_value.replace('million', '')) * 1e6
    elif 'billion' in cleaned_value:
        return float(cleaned_value.replace('billion', '')) * 1e9
    else:
        return float(cleaned_value)

# Apply the cleaning and conversion function to the 'Population' column
artist_df['US Units Sold'] = artist_df['US Units Sold'].apply(clean_and_convert)

artist_df = pd.DataFrame(artist_df)

# Remove the "million" from total claimed sales
artist_df['Total Claimed Sales'] = artist_df['Total Claimed Sales'].str.replace(' million', '').astype(float)

artist_df


,Artist,Country,Period active,Release-year of first charted record,Genre,US Units Sold,Total Claimed Sales
1,The Beatles,United Kingdom,1960–1970,1962,Rock / pop,217.25,600.0
2,Elvis Presley,United States,1953–1977,1956,Rock and roll / pop / country,199.65,500.0
3,Michael Jackson,United States,1964–2009,1971,Pop / rock / dance / soul / R&B,177.30,400.0
4,Elton John,United Kingdom,1962–present,1970,Pop / rock,138.05,300.0
5,Queen,United Kingdom,1971–present,1973,Rock,97.70,300.0
...,...,...,...,...,...,...,...
117,Bob Marley,Jamaica,1962–1981,1975,Reggae,21.85,75.0
118,The Police,United Kingdom,1977–19862007–2008,1978,Pop / rock,23.65,75.0
119,Barry Manilow,United States,1973–present,1973,Pop / soft rock,33.30,75.0
120,Kiss,United States,1972–present,1974,Hard rock / heavy metal,26.00,75.0
